<h1>AI-Exam</h1>
<p>For this exam project we decided to work with natural language processing, i.e text processing. We wanted to determaine if a strong correlation could be made between the overview/description of a movie and its genre. The overall idea was to make the genre of the movies our <i>dependable variable</i>, hence transforming all genres into numeric values, so that each number represents a different genre. Then, we would break the down the description of each movie, initially removing all stop words, i.e words that are concidered to have no siginificant or descriptive meaning of a text. Then check how frequently the remaining words appeared in each movie description, and assign them a "weight" accordingly. This way we would be able to predict the genre of a movie, based off of the movie description itself.</p>

<h2>The dataset</h2>
We've downloaded a dataset from kaggle, which can be found [here](https://www.kaggle.com/tmdb/tmdb-movie-metadata). The dataset contains movies from [IMDb](https://www.imdb.com/), and meta data about them. Most importantly it contains movies with their corresponding genres and a brief description about them. Initially we used another dataset where each movie was associated with one genre, but we got very poor results, and opted to change dataset, because we believed that the poor result primarily was a side effect of the size of the dataset. This was somewhat confirmed by the change of dataset, seeing as our models improved across the board after changing to a larger dataset. However the change did not come without issues. As briefly mentioned, in the initial dataset all movies were only related to <b>one</b> genre, whereas in the new dataset a movie could be related to <b>several</b> genres. This is an issue, as we only can have one dependable variable. To get around this issue, we decided that the first genre in the genre array, would be the movies genre, well knowning that the result potentially would be scewerd abit, as there's no guarentee the first genre in the array is the genre that fits the movie best. Furthermore it also means that our algorithm potentially guesses correct with the second or third, or even fourth genre, but it would still be classified as a false negative in our confusion matrix.

In [5]:
#Imports

import pandas as pd
import re #Regular expresion
import json
import numpy as np

import nltk #Natural language processing tool kit
from nltk.corpus import stopwords
nltk.download('stopwords')

# Modelling
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn import decomposition, ensemble

# Data preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Validation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mwe1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h2>Helper methods</h2>
<p>Three different helper methods has been developed during this project, to get the best possible results</p>

<h3>train_model</h3>
<p>The 'train_model' method was developed to get an easier overview, seeing as we're training a bunch of differnt models, which are all trained in the same way. Hence the method takes the following parameters: classfier, X_train, y_train and X_test. This allows to train several models in bulk, whilst maintaining an overview.</p>

<h3>convert_column_array_to_normal_array</h3>
<p>As the method name suggests, the method takes an array, which is shaped like so (x, 1), and converts to an array with the following shape (1, y). This is needed for some of the training models.</p>

<h3>transform_genre</h3>
<p>TBD</p>

In [6]:
# Helper methods
def train_model(classifier, X_train, y_train, X_test):
    
    # fit the training dataset on the classifier
    classifier.fit(X_train, y_train)
    
    # predict the labels on validation dataset
    pred = classifier.predict(X_test)

    return metrics.accuracy_score(y_test, pred)

def convert_column_array_to_normal_array(arr):
    vert_arr = []
    for i in arr:
        vert_arr.append(i)
    return vert_arr

def transform_genre(arr):
    arr = [[],[]]
    failedIndexes = []
    counter = 0
    while counter < len(arr):
        try:
            genreObject = array[counter]
            genreStr = genreObject[0]            
            genreStr = genreStr.replace('[', '')
            genreStr = genreStr.replace(']','')
            splitStr = genreStr.split('}, {')
            arr.append(counter)
            splitStr[0] = splitStr[0].replace('{', '')
            length = len(splitStr)
            splitStr[length-1] = splitStr[length-1].replace('}', '')
            for i in splitStr:
                i = "{"+i+"}"
                jsonobj = json.loads(i)
                genre = jsonobj["name"]
                arr[counter].append(genre)                      
            counter += 1 
        except:
            failedIndexes.sort()
            failedIndexes = failedIndexes[::-1] # reversing the list
            counter += 1
    return arr, failedIndexes

<h2>Data pre-processing</h2>
<p>Kaggle gave us the option to select which columns we would like to download, hence our dataset came with only two columns out of the box; genre and overview. Which means we didn't have to make this split programatically. As briefly touched upon in the introduction, a movie could have several genres, which was represented by a genre array. As an example a movie could be both a Sci-fi and an Action movie, which also seems rather logical. As we've also mentioned the order of the genres doesn't nessecarily mean anything in terms of genre specificity, and this is important, as it makes our results error-prone. We convert the ovewview and the genres into list which were previously stored in a dataframe.

Initally we create two empty lists, one list to hold all the genres that aren't empty, as well as a list to keep track of the indexes of genres that contain to data. We then start a while loop that iterates trough all the elements in our list of genres (y_list). We've defined a try/except clause, where, if possible, we store something we've called an "genreObject" which is actually a list of list only containing one element. Hence we grab that one element in the list, which we've named "genreStr", this element looks like a json object, but is in reality a poorly formatted string. We then contiue to split our poorly formatted string on white spaces, and grab the element at index position three, which is always the genre, given it's present. We then use regex to remove unwanted characters, which are included in our string, because as we mentioned, it looks like a json element, i.e we have curly braces and brackets in our stirng. As a biproduct of using regexs findall(), which return a list, we select the procssed string by selecting the element at index 0 the returned list, which always will be our genre. We then add that genre to a new list which we've called "y_", and finally we increment our counter, and repeat this process untill we have all of our genres. Given the genre array is empty we hit the except clause, where we store the index of the missing genre, and increment our and continue the while loop.

We now grab the list of failed indexes and sort, using pythons inbuilt sort() method, which is possible because it's numeric data. We then reverse our array which is super in easy in python using slicing. This is nessecary, otherwise when we start removing rows in our lists, the loop would crash itself, if we removed indexes from low to high. We now itterate trough our list of overviews and remove all of the overviews with the same index position as our missing genres, this is nessecary because the two list has to be same length.

Now we create an empty list called X_ and repeat the process mentioned above, now only with overviews instead. Afer that we stored X_ and y_ in X_list and y_list respectively, because we thought we were ready to fit our model with the data now. However we still had nan values in our X_list, so we converted X_lists to a dataframe, after its content had been formatted, and used isNull() which returns a bool, and stored those indexes that returned true, and removed them accordingly from both X_list and y_list.

Now were ready to fit and transom our model, so we split our datasets in test and traning batches and used fit_transform() on our labelenconder.</p>


In [7]:
moviesDF = pd.read_csv('tmdb_movies.csv') 

# RETURN TO THIS LATER
# moviesDF.groupby('genres').size() # prints how many of each genre there exists


y = moviesDF[['genres']] # our dependent variable

X = moviesDF[['overview']] # our independant variable

X_list = X.values.tolist()
y_list = y.values.tolist()

#result, failed = TransformGenre(y_list)

y_ = [] # temp, array for genres that arent empty
failedIndexes = [] # array to keep track of the index we had an empty genre, to be used later in deletion.

counter = 0
# we iterate over y_list to find all genres that arent empty, if one is empty its gonna trigger an exception
# which triggers our except clause. The except clause saved the index the error empty genre was located and progresses the counter
while counter < len(y_list):
    try:
        genreObject = y_list[counter]
        genreStr = genreObject[0]
        strstr = genreStr.split()
        indexedstr = strstr[3]
        finalStr = re.findall(r'\w+', indexedstr)    
        y_.append(finalStr[0])
        counter += 1 
    except:
        failedIndexes.append(counter)
        counter += 1
        
failedIndexes.sort()
failedIndexes = failedIndexes[::-1] # reversing the list
# as it turns out when you delete an index from a python list, it collapses the list, so we had to delete the highest index first to circumvent this.
for index in failedIndexes:
    del X_list[index]
    
y_list = y_ 

X_ =  [] # temp list to contain all strings
failedIndexes = []
counter = 0
# Currently X_list contains a collection of collections, this kind of list-ception is incompatible with the AI algorithm
# So we create this small loop to extract the string.
while counter < len(X_list):
    listLine = X_list[counter]
    listString = listLine[0]
    if not listString:
        failedIndexes.append(counter)
        counter += 1
        continue
    X_.append(listString)
    counter += 1
    
failedIndexes = failedIndexes[::-1]
for i in failedIndexes:
    del X_[i]
    
X_list = X_

# We had nan values in our X_list, we decided to convert the X_list back to a dataframe
# in order to run isnull(), which returns a list of false/true wether an entry is nan or not
# with this we simply saved the index of which the nan occured and deleted
# it from both our dependant and independant variable.

tempDf = pd.DataFrame(X_list)
boolList = tempDf.isnull().values

badIndexes = []
counter = 0
while counter < len(boolList):
    if boolList[counter][0]:
        badIndexes.append(counter)               
    counter += 1

badIndexes = badIndexes[::-1]

for i in badIndexes:
    del X_list[i]
    del y_list[i]
          
print(f'X_list length: {len(X_list)}, y_list length: {len(y_list)}')  
        
# we are splitting our dataset up here for training and later validation.
X_train, X_test, y_train, y_test = train_test_split(X_list,y_list)

print(f'{len(X_train)}, {len(X_test)}, {len(y_train)}, {len(y_test)}')

# Encoder to encode our dependant variable
encoder = preprocessing.LabelEncoder()


y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

X_list length: 4772, y_list length: 4772
3579, 1193, 3579, 1193


<h2>Feature Engineering</h2>
<p>Count Vector is a matrix notation of the dataset in which every row represents a document from the corpus, where corpus roughly translates to the training data. Every row represents a term from the corpus and has a corresponding value representing the frequency count of a particular term in a particular document. The count vectors have several different paramters which can be adjusted to yield different results, sometimes it's just about adjusting a paramter slightly, and suddenly results are amazing, but the same can be true in the oposit direction.</p>

<h3>analyzer</h3>
<p>analyzer is a paramter we set to equal to 'word'. Which means we only words, rather than if set it to 'char' we would concider every single char.</p>

<h3>stop_words</h3>
<p>We tried using the built in stopwords from the scikit toolkit, which can be accessed by setting the stop_words paramater equal to 'english'. Taking a closer look at the stopwords, we felt they were insufficient, and briefly we concidered making our own stopwords, untill we realized how much of a hassle that would be. Nltk, Natual Language Toolkit, also offers a list of stopwords, which we tried using, yieldig slightly better results, sometimes...<p/>

<h3>token_pattern</h3>
<p>Further more we tried adjusting the token pattern , which basically is a regular expression, expressing when a word is valid/to be concidered i.e having a token patter like: '\w{1,}', means that all words with more than one character is concidered. Where as if we change the pattern to '\w{3,}', we would only concider words with four or more characters. Finally we also tried to restricting the characters themselves, i.e adding the following to our token: '\w{x,}[a-z]' which essetianlly means we only concider character between and a-z, i.e not numbers or special characters. We played around with different combinations, but found that not providing a token at all yilded the most consistent results.<p/>
    
<h3>ngram_range</h3>    
<p>ngram_range defines how many words in a sentence are to be concidered as a 'unit'. Concider the following sentence: "The mighty lion sleeps tonight". If ngram_range was set to (1,2), our output of the sentence would be: "The":x, "mighty":x, ... "The mighty":x, "mighty lion":x, and so on. Adjusting this paramter clearly affected our results, and we would the most consistent result using ngram_range=(1,3). We opted our of using this paramter, because using analyzer='word' yileded better results.</p>
    
<h3>max_features</h3>
<p>Adjusting this setting change the number of total features we would concider, again a paramter that clearly affected our results, and essetially we stopped playing with it, and settled at 5000, where we seemed to get consistent results.</p>

In [75]:
#stop_words = set(stopwords.words('english'))
#token = 'r\w{4,}[a-z]'

count_vect = CountVectorizer(analyzer='word', stop_words='english', max_features=5000)
count_vect.fit(X_train)
# Now we are gonna transform the training and test data with our vectorized object

X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

# Now we are gonna use Term Frequency - Inverted Document Frequence (TF-IDF) vectors as features
# The score generate by the TF-IDF represents the relatuve importance of a term in a document and the entire corpus.
# We generate this score in two steps:
# The first computes the normalizeds term frequency (Tf) --- TF(x) = Number of times x appears in the document / total bynber if terms in the document. 
# the second computes the inverse document frequency  (IDF) --- IDF(x) = log_e(total number of documents / number of documents with term x in it)
# as mentioned earlier we could have chosen to use an n-gram composed of words, which we have implemented in line 61.
# now we are creating the TF-IDF score based on that n-gram.

#tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern='\w{1,}', max_features=5000)
tfidf_vect = TfidfVectorizer(encoding='utf-8',lowercase=True, stop_words='english', sublinear_tf=True, use_idf=True,max_features=5000)
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

<h2>Training and testing</h2>
<p>To train and test our models we used the helper method that we developed earlier on. To compare the results of the diferent models we printed out the accuracy of each model after it had run.</p>

<h3>Naive Bayers: Multinominal</h3>
<p>Naive bayers is a probolistic machine learning algorithm, that is particuallarly good at multiclassification. The algorithm is primarily based on bayers theorm which can be expressed as: P(A|B) = P(B|A)P(A)/P(B).
Naive bayers is often used solve several different AI tasks, but is often seen in text reconigition, for example; when it comes determaining if something is spam or not. The 'Naive' part comes from the assumption that the features are independant of each other which means that the pressence of one feature does not affect the pressence of another.</p>
    
<h3>Support Vector Machine</h3>
<p>A SVM (Support Vector Machine) looks at the outliers in a dataset, and tries to define a linear line, called the hyper plane, between two outliers to determine wether something can be classified as A or B. This is however often dificult to achieve on all datasets, hence the dimensional space is increased, untill a seperation can be made. There are several functions to achieve this effect, but overall it makes SVM rather costly to compute.<p>
    
<h3>Random Forest Classifier</h3>
<p>Random Forest Classifier can be used for both classification and regression tasks, but is generally concidered better for classifications tasks. The algorithm takes advantage of descission tree, and performs better the more data is available, but is also rather cost heavy because there's no pruning of the trees. Consider the following example; training_data = [1,2,3,4,5,6], then one of the trees (t) might be given the following data: t1 = [2,3,2,2,4,4], this ensures tree diversity, whilst still maintaining the data relevance. Furthermore random forest implements feature randomness, which is easier describe, by describing how a tree 'normally' decides its path. Before a tree decides its path, it has to concider all previous features. Random forest, takes a path based on a random subset of features. The algorithm, when used for classification, has each tree voting for an outcome, and the aggregated outcome with the highest vote will be the result, where as when used for regression, the average vote will be the result.<p>

In [74]:
label = convert_column_array_to_normal_array(y_train)

accuracy = train_model(classifier = naive_bayes.MultinomialNB(),
                       X_train = X_train_count,
                       y_train = label,
                       X_test = X_test_count)
print(f'NB, Count vectors: {accuracy}')

accuracy = train_model(classifier = naive_bayes.MultinomialNB(),
                       X_train = X_train_tfidf,
                       y_train = label,
                       X_test = X_test_tfidf)
print(f'NB, tf-idf vectors: {accuracy}')


accuracy = train_model(classifier = svm.SVC(),
                       X_train = X_train_count,
                       y_train = label,
                       X_test = X_test_count)
print(f'SVM, Count vectors: {accuracy}')

accuracy = train_model(classifier = svm.SVC(),
                       X_train = X_train_tfidf,
                       y_train = label,
                       X_test = X_test_tfidf)
print(f'SVM, tf-idf vectors: {accuracy}')

accuracy = train_model(classifier = ensemble.RandomForestClassifier(),
                       X_train = X_train_count,
                       y_train = label,
                       X_test = X_test_count)
print(f'RF, Count vectors: {accuracy}')

accuracy = train_model(classifier = ensemble.RandomForestClassifier(),
                       X_train = X_train_tfidf,
                       y_train = label,
                       X_test = X_test_tfidf)
print(f'RF, tf-idf vectors: {accuracy}')

NB, Count vectors: 0.40067057837384745
NB, tf-idf vectors: 0.3864207879295893
SVM, Count vectors: 0.38222967309304273
SVM, tf-idf vectors: 0.3889354568315172
RF, Count vectors: 0.366303436714166
RF, tf-idf vectors: 0.3688181056160939


<h2>Conclusion</h2>
<p>Several issues were raised during this project. Initially our dataset wasn't large enough to yield accurate results, hence we swapped to a larger dataset mid-project. The new dataset, as briefly touched upon in the introduction, contained several genres per movie, which raised new challenges. As we yet, don't know how to handle this problem, we opted to select the first out of n genres related to a movie, saying that, that one genre was the movies genre. This means our models might predict a genre, that in fact is related to the movie, according to the dataset, but is still clasified as a wrong prediction. As a result all of our models are rather inaccurate despite having, what we believe is, sufficient data preprocessing and model training.

Naive Bayers was the first algorithm we implemented, and because of the low accuracy score, we opted to implemented SVM, which yielded an equally low score. Finally we implemented Random Forest, which yielded the lowest and most varying result of them all. Playing with the different parameters, as describe in the Feature Engineering section, we were unsuccessful in improving the results more than the current implementation.

As a conclusion we believe that we've processed our data sufficiently, as well as trained our models correctly, but simply put we don't know how to handle muliple dependable variables.</p>

<h2>Group</h2>
<b>- Mikkel Wexøe Ertbjerg // cph-me209@cphbusiness.dk</b>

<b>- Nikolai Sjhøholm Christiansen // cph-nc103@cphbusiness.dk</b>

<b>- Nikolaj Dyring Jensen // cph-nj183@cphbusiness.dk</b>